# Week2_3 Assignment

## [BASIC](#Basic) 
- Custom Dataset 클래스를 구현할 수 있다.
- train_dataset, valid_dataset으로 데이터셋을 나눌 수 있다.


## [CHALLENGE](#Challenge)
- **dynamic padding**을 만드는 `collate_fn`을 구현할 수 있다. 
- `DataLoader` 클래스를 사용해 train_dataloaer와 valid_dataloader를 만들 수 있다.


## [ADVANCED](#Advanced)
- 기존 코드의 data_iterator를 data_loader로 대체해 학습을 실행할 수 있다.

### Reference
-[pytorch official document](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)   
-[collate_fn reference](https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/)   
-[dynamic padding explained](https://mccormickml.com/2020/07/29/smart-batching-tutorial/#dynamic-padding)   


In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

In [ ]:
# seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


## Basic

### 데이터 로드 및 결측치 제거 (복습)
- 지난 시간에 다운받은 데이터를 로드한다.
- 결측치 제거하고 `n_sample` 사이즈 만큼 샘플링해 새로운 데이터 프레임을 만든다. 단, label의 비율은 5:5로 샘플함에 유의한다.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/NLP강의/강의자료"

/content/drive/MyDrive/NLP강의/강의자료


In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "nsmc")

My current directory : /content/drive/MyDrive/NLP강의/강의자료


In [ ]:
df = pd.read_csv(os.path.join(_DATA_DIR,"ratings_train.txt"), delimiter="\t")

In [ ]:
# df에서 결측치 제거
df=df[~df.document.isna()]
df.shape

(149995, 3)

In [ ]:
# label별 데이터 수 확인
# 0 -> 부정 1 -> 긍정
df.label.value_counts() 

0    75170
1    74825
Name: label, dtype: int64

In [ ]:
def label_evenly_balanced_dataset_sampler(df, sample_size):
    """
    데이터 프레임의을 sample_size만큼 임의 추출해 새로운 데이터 프레임을 생성.
    이 때, "label"열의 값들이 동일한 비율을 갖도록(5:5) 할 것.
    """
    
    df = df.reset_index(drop=True) # Index로 iloc하기 위해서는 df의 index를 초기화해줘야 함
    
    neg_idx = df.loc[df.label==0].index
    neg_idx_sample = random.sample(neg_idx.to_list(), k=int(sample_size/2))

    pos_idx = df.loc[df.label==1].index
    pos_idx_sample = random.sample(pos_idx.to_list(), k=int(sample_size/2))

    return df.iloc[neg_idx_sample+pos_idx_sample]

In [ ]:
n_sample = 10000
sample_df = label_evenly_balanced_dataset_sampler(df, n_sample)
print(sample_df.shape)

NameError: name 'df' is not defined

### `CustomDataset `클래스 구현
- 클래스 정의
  - 생성자 입력 매개변수 
    - `input_data` : 리뷰 텍스트 리스트
    - `target_data` : 레이블 (0 또는 1) list
  - 생성자에서 생성할 변수
    - `X` : `input_data`
    - `Y` : `target_data`
  - 메소드 
    - `__len__()`
      - 데이터 총 개수를 반환
    - `__getitem__()`
      - 해당 인덱스의 (input_data, target_data) 튜플을 반환
  - 주의 사항
    - `torch.utils.data.Dataset()` 클래스를 부모 클래스로 상속받아 구현한다.  

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

In [ ]:
class CustomDataset(Dataset):
    """
    - input_data: list of string
    - target_data: list of int
    """
    
    def __init__(self, input_data:list, target_data:list) -> None:
        self.X = input_data
        self.Y = target_data
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
        
        

In [ ]:
dataset = CustomDataset(sample_df.document.to_list(), sample_df.label.to_list())

In [ ]:
# map-stype dataset 클래스는 indexing이 가능함
dataset[0]

('노래 두곡을 제외하고는 기억에 남는게 없다고 감히 말하고 싶다. 그냥 소소하게 볼만할수도 있지만 그 이상의 기대감은 큰 실마응로 돌아올수 있다....배우는 좋은데 스토리 구성 전개가 좀 아쉽다.',
 0)

In [ ]:
# 데이터 셋 총 개수 확인 가능
len(dataset)

11000

###  `torch.utils.data.random_split` 함수를 사용해 데이터셋을 train, valid로 분리
- 데이터 비율은 `train : valid = 9 : 1`으로 분리하라

In [ ]:
n_train = int(n_sample*0.9)
n_valid = int(n_sample*0.1)

In [ ]:
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

In [ ]:
len(train_dataset)

9900

In [ ]:
len(valid_dataset)

1100

## Challenge

### dynamic padding을 구현하는  `custom_collate_fn` 함수 구현 
- batch (`List[Tuple(input, target)]`)를 입력받아 토크나이즈한 후 텐서 형태로 변형해 반환 ( `Tuple(Tensor(tokenized_input), Tensor(target))`)하는 `collate_fn()` 함수를 구현하라. 
- 함수 정의
  - 입력 매개변수
    - `batch` : (input(string), target(int)) 튜플을 담고 있는 리스트.  만약 `batch_size`가 32라면 리스트는 32개의 튜플을 갖고 있다. 
  - 조건
    - input
      - [BERT Tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer) 클래스의 `__call__()` 메소드 사용 방법을 읽고, `__call__()` 파라미터를 조정해 dynamic padding을 구현한다.
      - 토크나이즈할 때 한 배치내 인풋들의 토큰 개수는 모두 동일할 수 있도록하라. 이때, 가장 긴 토큰을 가지고 있는 인풋을 기준으로 토큰 개수를 맞춘다. 즉, 토큰 개수가 모자란 인풋은 `[PAD]` 토큰을 추가한다. (이를 **dynamic padding**이라고 한다.) 
      - 토크나이저에서 반환된 값은 Tensor 타입이어야 한다. 
    - target
      - target은 Tensor 타입으로 변형한다.
  - 반환값
    - (tensorized_input, tensorized_label) 튜플

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [ ]:
def custom_collate_fn(batch):
    """
    - batch: list of tuples (input_data(string), target_data(int))
    
    한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
    이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
    
    한 배치 내 레이블(target)은 텐서화 함.
    
    (input, target) 튜플 형태를 반환.
    """
    input_list, target_list = [], []
    
    for _input, _target in batch:
        input_list.append(_input)
        target_list.append(_target)
    
    tensorized_input = tokenizer_bert(
        input_list,
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
    
    tensorized_label = torch.tensor(target_list)
    
    return tensorized_input, tensorized_label
    

### 위에서 구현한 `custom_collate_fn`함수를 적용해 DataLoader 인스턴스 생성
- `train_dataloader`
    - batch_size = 32
    - collate_fn = 위에서 구현한 함수
    - sampler = `RandomSampler()`
        - `train_dataset`의 학습 데이터를 셔플링 함
- `valid_dataloader`
    - batch_size = 64
    - collate_fn = 위에서 구현한 함수
    - sampler = `SequentialSampler()`
        - `valid_dataset`의 검증 데이터를 순차적으로 정렬함 (셔플X)

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size =16,
    sampler = RandomSampler(train_dataset),
    collate_fn = custom_collate_fn
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size =16,
    sampler = SequentialSampler(valid_dataset),
    collate_fn = custom_collate_fn
)

In [ ]:
# 배치마다 토큰 길이가 다른 것을 확인
for input_batch, target_batch in valid_dataloader:
    print(f"Batch input shape: {input_batch['input_ids'].shape}")
    print(f"Batch target shape: {target_batch.shape}")

Batch input shape: torch.Size([16, 77])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 56])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 37])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 41])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 60])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 46])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 28])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 42])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 60])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 75])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 30])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 63])
Batch target shape: torch.Size([16])
Batch input shape: torch.Size([16, 81])
Batch target shape: torch.Size([16])

## Advanced

### 어제 생성한 `train()` 함수의 입력값이었던 `data_iterator`를  `data_loader`로 변경

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

In [ ]:
def train(model, data_loader):

        # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
        total_loss, batch_loss, batch_count = 0,0,0
        
        # model을 train 모드로 설정 & device 할당
        model.train()
        model.to(device)
        
        # data iterator를 돌면서 하나씩 학습
        for step, batch in enumerate(data_loader):
            batch_count+=1
            
            # tensor 연산 전, 각 tensor에 device 할당
            batch = tuple(item.to(device) for item in batch)
            
            batch_input, batch_label = batch
            
            # batch마다 모델이 갖고 있는 기존 gradient를 초기화
            model.zero_grad()
            
            # forward
            logits = model(**batch_input)
            
            # loss
            loss = loss_fct(logits, batch_label)
            batch_loss += loss.item()
            total_loss += loss.item()
            
            # backward -> 파라미터의 미분(gradient)를 자동으로 계산
            loss.backward()
            
            # optimizer 업데이트
            optimizer.step()
            
            # 배치 10개씩 처리할 때마다 평균 loss를 출력
            if (step % 10 == 0 and step != 0):
                print(f"Step : {step}, Avg Loss : {batch_loss / batch_count:.4f}")
                
                # 변수 초기화 
                batch_loss, batch_count = 0,0
        
        print(f"Mean Loss : {total_loss/(step+1):.4f}")
        print("Train Finished")

### 지금까지 구현한 함수와 클래스를 모두 불러와 `train()` 함수를 실행하자
- fine-tuning 모델 클래스 (`CustomClassifier`)
    - hidden_size = 768
    - n_label = 2
- 데이터 이터레이터 함수 (`data_iterator`)
    - batch_size = 32
- loss 
    - `CrossEntropyLoss()`
- optimizer
    - `AdamW()`
    - lr = 2e-5


In [ ]:
class CustomClassifier(nn.Module):

    def __init__(self, hidden_size: int, n_label: int):
        super(CustomClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("klue/bert-base")

        dropout_rate = 0.1
        linear_layer_hidden_size = 32

        self.classifier = nn.Sequential(
        nn.Linear(hidden_size, linear_layer_hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(linear_layer_hidden_size, n_label)
        )

    

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        
        # BERT 모델의 마지막 레이어의 첫번재 토큰을 인덱싱
        last_hidden_states = outputs[0] # last hidden states (batch_size, sequence_len, hidden_size)
        cls_token_last_hidden_states = last_hidden_states[:,0,:] # (batch_size, first_token, hidden_size)

        logits = self.classifier(cls_token_last_hidden_states)

        return logits




In [ ]:
# 모델
model = CustomClassifier(hidden_size=768, n_label=2)

# 데이터로더
batch_size = 32
train_dataloader = DataLoader(
    train_dataset,
    batch_size =batch_size,
    sampler = RandomSampler(train_dataset),
    collate_fn = custom_collate_fn
)

# 로스 및 옵티마이저
loss_fct = CrossEntropyLoss()
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8
)

# 학습 시작
train(model, train_iterator)